In [1]:
#code for requesting all bands for for multiple days (multiple timestamp) for ROME
import datetime
import os
import getpass

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# Step 1: Set up your configuration
config = SHConfig()
config.sh_client_id = 'sh-f4a2dffd-e52c-4d3b-9e5b-e1a2787df9b6'  # Replace with your Sentinel Hub client ID
config.sh_client_secret = 'hUZYxAyDlH9WEb3GTnLvowHkJLnBkios'  # Replace with your Sentinel Hub client secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"


#betsiboka_coords_wgs84 = (46.16, -16.15, 46.51, -15.58)
betsiboka_coords_wgs84 = ( 12.36,41.89,12.46,41.99)
resolution = 10
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")


start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2019, 12, 30)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B04"]
            }],
            output: {
                bands: 1
            }
        };
    }

    function evaluatePixel(sample) {
        return [ sample.B04];
    }
"""

request_true_color = SentinelHubRequest(
    data_folder='test_dir',
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2018-01-01", "2018-12-31"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        data_folder="test_dir",
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C.define_from(
                    "s2l1c", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )
# create a list of requests
list_of_requests = [get_true_color_request(slot).get_data(save_data=True) for slot in slots]

/home/aneesha/miniconda3/envs/isce_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Image shape at 10 m resolution: (863, 1085) pixels
Monthly time windows:

('2019-01-01', '2019-01-28')
('2019-01-28', '2019-02-25')
('2019-02-25', '2019-03-25')
('2019-03-25', '2019-04-22')
('2019-04-22', '2019-05-20')
('2019-05-20', '2019-06-17')
('2019-06-17', '2019-07-15')
('2019-07-15', '2019-08-12')
('2019-08-12', '2019-09-09')
('2019-09-09', '2019-10-07')
('2019-10-07', '2019-11-04')
('2019-11-04', '2019-12-02')


In [2]:
pip install matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 3.4 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 8.0 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 7.8 MB/s eta 0:00:009.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 8.5 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.6 MB/s eta 0:00:008 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 14.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import rasterio

# Open the GeoTIFF file
with rasterio.open('response.tiff') as src:
    # Read all bands
    bands_data = src.read()

    # Get metadata
    metadata = src.meta

# Extract each band into separate variables
band1 = bands_data[0]  # Assuming bands are indexed from 0
band2 = bands_data[1]
band3 = bands_data[2]

# Save each band as a separate GeoTIFF file
with rasterio.open('band1.tif', 'w', **metadata) as dst1:
    dst1.write(band1, 1)

with rasterio.open('band2.tif', 'w', **metadata) as dst2:
    dst2.write(band2, 1)

with rasterio.open('band3.tif', 'w', **metadata) as dst3:
    dst3.write(band3, 1)

In [7]:

#code for requesting all bands for for multiple days (multiple timestamp) for ROME - to calculate the buildup index fro the urban development change detection
import datetime
import os
import getpass

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# Step 1: Set up your configuration
config = SHConfig()
config.sh_client_id = 'sh-f4a2dffd-e52c-4d3b-9e5b-e1a2787df9b6'  # Replace with your Sentinel Hub client ID
config.sh_client_secret = 'hUZYxAyDlH9WEb3GTnLvowHkJLnBkios'  # Replace with your Sentinel Hub client secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"


#betsiboka_coords_wgs84 = (46.16, -16.15, 46.51, -15.58)
betsiboka_coords_wgs84 = ( 12.36,41.87,12.48,41.99)
resolution = 10
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")


start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2020, 12, 30)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)
evalscript_true_color = """
    //VERSION=3

function setup() {
    return {
        input: [{
            bands: ["B04", "B05", "B06"]
        }],
        output: {
            bands: 1
        }
    };
}

function evaluatePixel(sample) {
    // Extract bands
    var B04 = sample.B04;
    var B05 = sample.B05;
    var B06 = sample.B06;

    // Calculate NDVI
    var ndvi = (B05 - B04) / (B05 + B04);

    // Calculate NDBI
    var ndbi = (B06 - B05) / (B06 + B05);

    // Calculate BU
    var BU = ndvi - ndbi;

    // Return the result
    return [BU];
 }
"""

request_true_color = SentinelHubRequest(
    data_folder='test_dir',
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2020-01-01", "2020-12-31"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        data_folder="test_dir",
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C.define_from(
                    "s2l1c", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )
# create a list of requests
list_of_requests = [get_true_color_request(slot).get_data(save_data=True) for slot in slots]

Image shape at 10 m resolution: (1035, 1303) pixels
Monthly time windows:

('2020-01-01', '2020-01-29')
('2020-01-29', '2020-02-26')
('2020-02-26', '2020-03-25')
('2020-03-25', '2020-04-22')
('2020-04-22', '2020-05-20')
('2020-05-20', '2020-06-17')
('2020-06-17', '2020-07-15')
('2020-07-15', '2020-08-12')
('2020-08-12', '2020-09-09')
('2020-09-09', '2020-10-07')
('2020-10-07', '2020-11-04')
('2020-11-04', '2020-12-02')
